In [1]:
import numpy as np, pandas as pd, warnings, os, openai, json
# from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
# import requests
# from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
# import pickle
# from sklearn.metrics.pairwise import cosine_similarity
# from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import 
# from langchain.schema import Sys
from langgraph.graph import StateGraph, END
from chromadb import Documents, EmbeddingFunction, Embeddings
import chromadb
from typing import TypedDict, Literal, Dict, Any, Optional


In [19]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

# GPT.ver

In [10]:
def invoke(
        #    prompt,
           system_prompt,
           user_text,
           api_key=api_key,
           model="gpt-4.1",
           temperature=0):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_text},
        ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        # messages=prompt,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    # return stream
    return getattr(stream, "content", stream)

memory = ConversationBufferMemory(return_messages=True)

In [20]:
# =========================
# 3) LLM 호출부 (플러그형)
# =========================
# 아래는 "어떤 LLM을 쓰든" 교체 가능하도록 어댑터 형태로 작성.
# 예: langchain_openai.ChatOpenAI, Azure, Ollama, etc.

class LLMClient:
    """아주 간단한 LLM 인터페이스(교체용)."""
    def __init__(self, model, temperature=0, api_key=None):
        self.model = model
        self.temperature = temperature
        self.api_key = api_key
        

    def invoke(self, system_prompt: str, user_text: str) -> str:
        # LangChain ChatModel 스타일을 가정한 예시.
        # 사용하는 모델에 맞게 수정하면 됨.
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_text},
        ]
        client = OpenAI(api_key=api_key)
        stream = client.chat.completions.create(
        model=self.model,
        messages=messages,
        # messages=prompt,
        temperature=self.temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )
        # resp = self.model.invoke(messages)
        # resp가 문자열이거나 AIMessage일 수 있으니 안전 처리
        return getattr(stream.choices[0].message, "content", stream.choices[0].message)

# SentenceTransformer Function

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3", )


# 과목 분류 코드

In [13]:
Specialty = Literal["산부인과", "소아청소년과", "응급의학과", "내과", "일반"]

class GraphState(TypedDict, total=False):
    question: str
    specialty: Specialty
    answer: str
    debug: Dict[str, Any]


SPECIALTY_SYSTEM_PROMPTS: Dict[Specialty, str] = {
    "산부인과": (
        "당신은 산부인과 전문 지식에 기반해 환자 질문에 답하는 의료 상담 도우미입니다.\n"
        "- 임신 가능성, 생리/부정출혈, 골반통, 질 분비물/가려움, 성병 가능성 등 고려\n"
        "- 위험 신호(심한 출혈, 실신, 극심한 통증, 임신 중 출혈 등) 시 응급 권고\n"
        "- 진단이 아닌 정보 제공으로, 필요한 경우 병원 방문/검사 권고\n"
        "출력 형식:\n"
        "1) 가능한 원인(우선순위)\n2) 추가로 확인할 질문\n3) 집에서 할 수 있는 조치\n4) 즉시 진료/응급실이 필요한 경우\n"
    ),
    "소아청소년과": (
        "당신은 소아청소년과 상담 도우미입니다.\n"
        "- 연령/체중/접종력/기저질환 고려\n"
        "- 탈수, 호흡곤란, 고열 지속, 의식 저하 등 위험 신호 강조\n"
        "출력 형식:\n"
        "1) 가능성 높은 원인\n2) 집에서 관찰 포인트\n3) 병원 방문 기준(응급 포함)\n"
    ),
    "응급의학과": (
        "당신은 응급의학과 트리아지(중증도 판단) 중심 상담 도우미입니다.\n"
        "- 생명 위협/즉시 처치가 필요한 위험 신호를 먼저 확인\n"
        "- 빨리 병원/응급실이 필요한 경우를 명확히 안내\n"
        "출력 형식:\n"
        "1) 지금 당장 확인할 위험 신호 체크리스트\n2) 응급실/119 기준\n3) 응급이 아닐 때 다음 단계\n"
    ),
    "내과": (
        "당신은 내과 상담 도우미입니다.\n"
        "- 복통, 소화기, 감염, 만성질환(당뇨/고혈압 등) 관점에서 설명\n"
        "- 감별진단은 단정하지 말고 가능성/다음 검사/경고증상 중심\n"
        "출력 형식:\n"
        "1) 가능한 원인(우선순위)\n2) 추가 질문(증상/기간/동반증상)\n"
        "3) 자가 관리\n4) 진료 필요 기준(응급 포함)\n"
    ),
    "일반": (
        "당신은 일반 의료 정보 제공 도우미입니다.\n"
        "- 전문과 판별이 어려운 경우에도 안전하게 안내\n"
        "- 경고 증상과 다음 행동(어떤 과로 갈지)을 제시\n"
        "출력 형식:\n"
        "1) 가능성/설명\n2) 추가 질문\n3) 다음 행동(어떤 과/검사)\n4) 위험 신호\n"
    ),
}

# 분류 방식 1 : 키워드 기반 분류

In [14]:
def classify_specialty_keyword(state: GraphState) -> GraphState:
    q = state["question"].strip()

    # 매우 단순한 예시 규칙(필요하면 확장)
    obgyn_kw = ["임신", "생리", "월경", "부정출혈", "질", "냉", "골반", "자궁", "난소", "피임"]
    peds_kw = ["아기", "신생아", "유아", "어린이", "초등", "청소년", "아이", "소아", "예방접종"]
    er_kw = ["호흡곤란", "흉통", "의식", "실신", "경련", "피토", "토혈", "혈변", "심한 출혈", "마비", "극심한 통증"]
    im_kw = ["복통", "설사", "변비", "속쓰림", "구토", "발열", "기침", "가래", "피로", "어지럼", "소화", "위", "장"]

    def has_any(keywords):
        return any(k in q for k in keywords)

    if has_any(er_kw):
        specialty: Specialty = "응급의학과"
    elif has_any(obgyn_kw):
        specialty = "산부인과"
    elif has_any(peds_kw):
        specialty = "소아청소년과"
    elif has_any(im_kw):
        specialty = "내과"
    else:
        specialty = "일반"

    return {
        **state,
        "specialty": specialty,
        "debug": {**state.get("debug", {}), 
                  "classifier": "keyword",
                  "matched_specialty": specialty},
    }



# 분류 방식 2 : LLM 기반 분류

In [15]:
# =========================
# 5) 분류기 노드: (B) LLM 기반(권장: 정확도↑)
# =========================
def make_classify_specialty_llm(llm: LLMClient):
    system = (
        "너는 의료 질문을 진료과로 라우팅하는 분류기다.\n"
        "가능한 라벨은 정확히 다음 5개만 사용한다:\n"
        "- 산부인과\n- 소아청소년과\n- 응급의학과\n- 내과\n- 일반\n\n"
        "규칙:\n"
        "1) 응급 신호(의식저하, 호흡곤란, 흉통, 심한 출혈 등)가 있으면 응급의학과 우선\n"
        "2) 임신/생리/질/골반 통증 등은 산부인과\n"
        "3) 소아/청소년(아이, 아기 등)은 소아청소년과\n"
        "4) 복통/소화기/감염/만성질환 중심은 내과\n"
        "5) 애매하면 일반\n\n"
        "출력은 라벨 1개만, 다른 문장 금지."
    )

    def _node(state: GraphState) -> GraphState:
        q = state["question"].strip()
        raw = llm.invoke(system, q).strip()

        # 안전하게 라벨만 정규화
        label_map = {
            "산부인과": "산부인과",
            "소아청소년과": "소아청소년과",
            "응급의학과": "응급의학과",
            "내과": "내과",
            "일반": "일반",
        }
        specialty: Specialty = label_map.get(raw, "일반")

        return {
            **state,
            "specialty": specialty,
            "debug": {**state.get("debug", {}), "classifier": "llm", "raw": raw, "normalized": specialty},
        }

    return _node



In [41]:


# =========================
# 6) Specialty 응답 노드 (프롬프트 분기)
# =========================
def make_specialty_answer_node(specialty: Specialty, llm: LLMClient):
    def _node(state: GraphState) -> GraphState:
        q = state["question"].strip()
        sys_prompt = SPECIALTY_SYSTEM_PROMPTS[specialty]
        
        answer = llm.invoke(sys_prompt, q)

        return {
            **state,
            "answer": answer,
            # "debug": {**state.get("debug", {}), "classifier": "llm", "raw": raw, "normalized": specialty},

        }
    return _node


# =========================
# 7) Router 함수: conditional edge에서 사용
# =========================
def route_by_specialty(state: GraphState) -> Specialty:
    # 분류 결과가 없거나 이상하면 일반으로
    sp = state.get("specialty", "일반")
    if sp not in ("산부인과", "소아청소년과", "응급의학과", "내과", "일반"):
        return "일반"
    return sp  # type: ignore


# =========================
# 8) 그래프 빌드 함수
# =========================
def build_graph(llm_model,api_key, use_llm_classifier: bool = False):
    """
    llm_model: LangChain ChatModel(예: ChatOpenAI, OllamaChat 등) 객체
    use_llm_classifier: True면 LLM 기반 분류기 사용, False면 키워드 분류기 사용
    """
    llm = LLMClient(model=llm_model, api_key=api_key)

    g = StateGraph(GraphState)

    # 분류 노드 선택
    if use_llm_classifier:
        print("Use LLM Classifier")
        g.add_node("classify_specialty", make_classify_specialty_llm(llm))
    else:
        print("Use Keyword Classifier")
        g.add_node("classify_specialty", classify_specialty_keyword)

    # 각 분기 노드
    g.add_node("산부인과", make_specialty_answer_node("산부인과", llm))
    g.add_node("소아청소년과", make_specialty_answer_node("소아청소년과", llm))
    g.add_node("응급의학과", make_specialty_answer_node("응급의학과", llm))
    g.add_node("내과", make_specialty_answer_node("내과", llm))
    g.add_node("일반", make_specialty_answer_node("일반", llm))

    # Entry
    g.set_entry_point("classify_specialty")

    # Conditional routing
    g.add_conditional_edges(
        "classify_specialty",
        route_by_specialty,
        {
            "산부인과": "산부인과",
            "소아청소년과": "소아청소년과",
            "응급의학과": "응급의학과",
            "내과": "내과",
            "일반": "일반",
        },
    )

    # 각 노드는 END로
    g.add_edge("산부인과", END)
    g.add_edge("소아청소년과", END)
    g.add_edge("응급의학과", END)
    g.add_edge("내과", END)
    g.add_edge("일반", END)

    return g.compile()


# =========================
# 9) 사용 예시
# =========================
# if __name__ == "__main__":
#     # 예시: ChatOpenAI를 쓴다면 (사용 환경에 맞게 교체)
#     # from langchain_openai import ChatOpenAI
#     # model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#     # 여기서는 model 변수만 "네가 쓰는 LLM"로 넣어주면 됨.
#     model = None  # TODO: 네 LLM 객체로 교체

#     app = build_graph(model, use_llm_classifier=False)

#     input_state: GraphState = {"question": "내가 복통이 있는데 왼쪽 아랫배가 아파."}
#     result = app.invoke(input_state)

#     print("specialty =", result.get("specialty"))
#     print("answer =", result.get("answer"))
#     print("debug =", result.get("debug"))


In [44]:
app = build_graph(
    llm_model="gpt-4.1",
    api_key=api_key,
    use_llm_classifier=False)

Use Keyword Classifier


In [45]:
result = app.invoke({"question": "내가 복통이 있는데 왼쪽 아랫배가 아파."})

In [46]:
result

{'question': '내가 복통이 있는데 왼쪽 아랫배가 아파.',
 'specialty': '내과',
 'answer': '1) 가능한 원인(우선순위)  \n왼쪽 아랫배(좌하복부) 통증의 흔한 원인으로는 다음이 있습니다.\n- **장 관련 질환**: 대장(특히 S자 결장) 염증(게실염), 과민성 장증후군, 변비, 장 감염 등\n- **비뇨기계**: 요로감염, 신장결석(요관 결석)\n- **부인과적 원인(여성의 경우)**: 난소낭종, 난소염전, 자궁외임신, 생리 관련 통증 등\n- **기타**: 근육통, 탈장 등\n\n2) 추가 질문(증상/기간/동반증상)\n- 통증이 시작된 시기와 양상(갑자기/서서히, 쥐어짜는/찌르는 등)\n- 발열, 오한, 구역/구토, 설사, 변비, 혈변, 소변 이상(빈뇨, 배뇨통, 혈뇨)\n- 최근 식사, 여행, 항생제 복용력\n- 여성의 경우: 마지막 생리일, 임신 가능성, 질 출혈/분비물\n- 과거 비슷한 증상 경험, 기저질환(당뇨, 고혈압, 장질환 등)\n\n3) 자가 관리\n- 일단 자극적인 음식, 카페인, 알코올 피하기\n- 충분한 수분 섭취, 휴식\n- 복부 온찜질(심한 염증이 의심되면 피함)\n- 증상 관찰 및 악화 시 기록\n\n4) 진료 필요 기준(응급 포함)\n- **즉시 응급실 방문**:  \n  - 극심한 통증, 점점 심해지는 통증\n  - 지속적 구토, 혈변/흑색변, 소변이 전혀 안 나옴\n  - 고열(38.5도 이상), 오한, 의식 저하\n  - 복부가 딱딱해지거나 만지면 심하게 아픔\n  - 임신 가능성이 있는 여성에서 심한 통증, 질 출혈\n\n- **외래 진료 권고**:  \n  - 통증이 1~2일 이상 지속되거나 반복됨\n  - 미열, 소화불량, 배변/배뇨 변화 동반\n  - 기존 만성질환(당뇨, 신장질환 등)이 있는 경우\n\n추가 증상이나 궁금한 점이 있으면 말씀해 주세요.',
 'debug': {'classifier': 'keyword', 'matched_specialty': '내과'}}

In [21]:
app = build_graph(invoke, use_llm_classifier=True)

In [18]:
input_state: GraphState = {"question": "내가 복통이 있는데 왼쪽 아랫배가 아파."}

app.invoke(input_state)

내가 복통이 있는데 왼쪽 아랫배가 아파.


TypeError: unhashable type: 'ChatCompletion'

# langgraph Chart 저장

In [ ]:
png = app.get_graph().draw_mermaid_png()
with open("graph.png", "wb") as f:
    f.write(png)

In [16]:
input_state: GraphState = {"question": "내가 복통이 있는데 왼쪽 아랫배가 아파."}

In [6]:
question = '배속이 너무 아프고 땀이 너무 나.'
messages = [
                    {"role": "system", 
                        "content": f"""당신은 의료분야의 전문 지식을 갖춘 병원 안내데스크 직원입니다.
                        현재 환자가 들어와 간단한 질의에 대하여 상담을 받고 있습니다. 저희 병원에서 받고있는 과는 '산부인과','소아청소년과','응급의학과','내과'입니다.
                        해당 과에 대한 질문이 들어온다면 해당 과를 뱉어주세요.
                        만약 제시한 과 중 어느 곳에도 속하지 않는다면 'General'을 반환해줘.
                        산부인과 관련 질문이 들어오면 '산부인과', 소아청소년과 관련 질문이 들어오면 '소아청소년과', 응급의학과 관련 질문이 들어오면 '응급의학과', 내과 관련 질문이 들어오면 '내과' 
                        너가 답변할 수 있는 내용은 '산부인과','내과','응급의학과','소아청소년과','General'] 뿐이야.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ]

In [8]:
response = invoke(messages)

In [12]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model: str = "bge-m3"):
        self.model = model
        self.embedding_model = OllamaEmbeddings(model=self.model,
                                                # base_url = base_url,
                                                )

    def __call__(self, input: Documents) -> Embeddings:
        # input은 List[str] 형태로 들어옴
        if isinstance(input, str):
            input = [input]

        embeddings = [self.embedding_model.embed_query(doc) for doc in input]
        return np.array(embeddings).tolist()  # ChromaDB는 List[List[float]] 기대
embedding_fn = MyEmbeddingFunction(model="bge-m3")

# ChromaDB 클라이언트
db_path = "../data/chroma_db"
client = chromadb.PersistentClient(path =db_path)

#collection 삭제
# client.delete_collection('medical_qa')

# Collection 생성
collection = client.get_or_create_collection('medical_qa',
                                             embedding_function=embedding_fn
                                             )


In [ ]:
collection.query(
    query_texts=["신생아 중환자실에서 기계환기를 받고 있는"],
    n_results=3,
    where={"domain":15}
    
    
)

In [14]:
collection = client.get_collection('medical_qa')

In [32]:
# 3️⃣ 1차 + 2차 필터링 함수
def chroma_query_with_secondary_filter(domain_value, q_type_value, n_results=100):
    # 1차 필터링 (ChromaDB에서 domain 기준으로 검색)
    results = collection.query(
        query_texts=[""],  # 검색어 없을 때는 전체 쿼리
        where={"domain": domain_value},
        n_results=n_results
    )

    # 2차 필터링 (로컬 Python에서 q_type 기준 필터링)
    filtered_results = []
    for i in range(len(results["ids"][0])):
        meta = results["metadatas"][0][i]
        if meta.get("q_type") == q_type_value:
            filtered_results.append({
                "id": results["ids"][0][i],
                "text": results["documents"][0][i],
                "metadata": meta
            })

    return filtered_results

In [33]:
filtered = chroma_query_with_secondary_filter(domain_value=17, q_type_value=1)

for item in filtered:
    print(item)

/home/jellybee/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:25<00:00, 3.31MiB/s]


InvalidArgumentError: Collection expecting embedding with dimension of 1024, got 384

In [ ]:
# results = collection.get(
#     where={
#         "domain": {"$gt": 17},
#            "q_type": {"$gt": 10}
#            }  # 예: 메타데이터 중 category가 'news'인 문서
# )
collection = client.get_collection('medical_qa')
results = collection.get(
    where={
        "domain": {"$gt": 17},
        "q_type": {"$gt": 10}
    }
)


ValueError: Expected where to have exactly one operator, got {'domain': {'$gt': 17}, 'q_type': {'$gt': 10}} in get.

In [24]:
results

{'ids': ['필수_629424.json',
  '필수_615373.json',
  '필수_612657.json',
  '필수_67612.json',
  '필수_68520.json',
  '필수_66870.json',
  '필수_627007.json',
  '필수_620065.json',
  '필수_612058.json',
  '필수_65870.json',
  '필수_626056.json',
  '필수_69892.json',
  '필수_65952.json',
  '필수_616339.json',
  '필수_618540.json',
  '필수_65249.json',
  '필수_65454.json',
  '필수_68469.json',
  '필수_68355.json',
  '필수_613981.json',
  '필수_6175.json',
  '필수_5289.json',
  '필수_65077.json',
  '필수_62506.json',
  '필수_624826.json',
  '필수_621204.json',
  '필수_65239.json',
  '필수_615266.json',
  '필수_4258.json',
  '필수_615156.json',
  '필수_628625.json',
  '필수_5359.json',
  '필수_619689.json',
  '필수_67099.json',
  '필수_1140.json',
  '필수_619645.json',
  '필수_61694.json',
  '필수_610188.json',
  '필수_624264.json',
  '필수_628843.json',
  '필수_68429.json',
  '필수_4123.json',
  '필수_628457.json',
  '필수_63552.json',
  '필수_612212.json',
  '필수_629259.json',
  '필수_619318.json',
  '필수_61346.json',
  '필수_177.json',
  '필수_2852.json',
  '필수_62927.json',
  '필수_622

In [17]:
file_path = '../../../medical_data/02.라벨링데이터/TL_내과/필수_11.json'
with open(file_path, 'r', encoding='utf-8-sig') as f:
            json_file = f.read()
            s1= json.loads(json_file)

In [18]:
s1

{'qa_id': 11,
 'domain': 17,
 'q_type': 1,
 'question': '23세 여자가 3개월 전부터 기침을 한다며 내원했다. 기침은 밤에 누워 자려고 할 때 심해진다고 한다. 1년 전에도 같은 시기에 기침이 3개월 동안 지속되다가 저절로 호전된 병력이 있다. 콧물이나 인후부 불편감은 없으며, 비흡연자이고 복용 중인 약물도 없다. 신체검사에서 혈압 120/80mmHg, 맥박 78회/분, 호흡 18회/분, 체온 36.5°C로 측정되었다. 청진상 호흡음은 정상이었고, 가슴 X선 사진과 코곁굴 X선 사진에서도 이상 소견이 없었다. 폐기능검사 결과는 다음과 같다.  \n- 강제 폐활량(FVC): 정상 예측치의 91%  \n- 1초간 강제날숨유량(FEV1): 정상 예측치의 85%  \n- 1초간 강제날숨유량/강제폐활량(FEV1/FVC): 75%  \n\n이 환자에서 다음으로 시행해야 할 검사는 무엇인가?  \n1) 기관지내시경  \n2) 기관지 확장제 반응 검사  \n3) 가슴 컴퓨터단층촬영(CT) \n4) 메타콜린 기관지유발검사  \n5) 폐 확산능 검사',
 'answer': '4) 메타콜린 기관지유발검사'}

In [16]:
pd.read_json('../../../medical_data/02.라벨링데이터/TL_내과/필수_11.json')

ValueError: Expected object or value

In [15]:
pwd

'/home/jellybee/github/LLM_Research/Langgraph_Research/script'

In [16]:
def generate_response(state, category):
    history = memory.chat_memory.messages  # 이전 대화
    current_question = state["question"]
    
    # 현재 질문과 카테고리에 맞는 프롬프트 구성
    prompt = f"사용자가 이렇게 질문했습니다: '{current_question}'\n"
    # if category == "Normal":
    #     prompt += "→ 질문에 대해서 개념 및 풀이과정을 상세히 설명해주세요."
    if category == "산부인과":
        prompt += "→ 산부인과 질의입니다. 당신은 산부인과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    elif category == "소아청소년과":
        prompt += "→ 소아청소년과 질의입니다. 당신은 소아청소년과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    elif category == "응급의학과":
        prompt += "→ 응급의학과 질의입니다. 당신은 응급의학과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    elif category == "내과":
        prompt += "→ 내과 질의입니다. 당신은 내과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    else:
        prompt += "→ 사용자의 물음에 친절히 답해주세요."
    
    # 모델 호출
    answer = invoke(history + [{"role": "user", "content": prompt}])
    response = answer.choices[0].message.content
    
    # 응답을 메모리에 저장
    memory.chat_memory.add_user_message(current_question)
    memory.chat_memory.add_ai_message(response)
    
    state["response"] = response
    return state

def decide_next(state):
    return state["type"]

In [17]:
# Node Wrappers
baby_node = RunnableLambda(lambda s: generate_response(s, "산부인과"))
kid_node = RunnableLambda(lambda s: generate_response(s, "소아청소년과"))
emergency_node = RunnableLambda(lambda s: generate_response(s, "응급의학과"))
internal_node = RunnableLambda(lambda s: generate_response(s, "내과"))
general_node = RunnableLambda(lambda s: generate_response(s, "General"))

In [18]:
# ✅ LangGraph 구성
state_type = dict
graph = StateGraph(state_schema=state_type)

graph.add_node("classify", RunnableLambda(classify_question))
graph.add_node("산부인과", baby_node)
graph.add_node("소아청소년과", kid_node)
graph.add_node("응급의학과", emergency_node)
graph.add_node("내과", internal_node)
graph.add_node("General", general_node)


graph.add_edge("산부인과", END)
graph.add_edge("소아청소년과", END)
graph.add_edge("응급의학과", END)
graph.add_edge("내과", END)
graph.add_edge("General", END)


In [19]:
graph.set_entry_point("classify")

graph.add_conditional_edges(
    "classify",
    path=RunnableLambda(decide_next),
    path_map={
        "산부인과": "산부인과",
        "소아청소년과": "소아청소년과",
        "응급의학과": "응급의학과",
        "내과": "내과",
        "General": "General",
    
    }
)

In [20]:
chat_agent = graph.compile()


In [21]:
chat_agent.invoke({"question": '배속이 너무 아프고 땀이 너무 나.'})


{'question': '배속이 너무 아프고 땀이 너무 나.',
 'type': '내과',
 'response': '안녕하세요. 말씀해주신 "배가 너무 아프고 땀이 너무 난다"는 증상은 여러 가지 원인에 의해 발생할 수 있습니다. 내과적으로 중요한 증상일 수 있으니, 아래 내용을 참고해주시고 필요시 빠르게 진료를 받으시길 권유드립니다.\n\n### 1. 증상의 원인\n배가 아프고 땀이 나는 증상은 다음과 같은 다양한 원인에 의해 발생할 수 있습니다.\n\n- **급성 복통(급성 복부질환)**: 맹장염(충수염), 담낭염, 췌장염, 장폐색, 위궤양 천공 등\n- **소화기계 감염**: 식중독, 바이러스성 장염 등\n- **심혈관계 문제**: 심근경색(특히 상복부 통증으로 나타날 수 있음)\n- **내분비 문제**: 저혈당, 갑상선 기능 항진증 등\n- **기타**: 신장결석, 요로감염, 산통 등\n\n### 2. 위험 신호(경고 증상)\n아래와 같은 증상이 동반된다면 즉시 응급실 방문이 필요합니다.\n\n- 통증이 갑자기 심해지거나, 참을 수 없을 정도로 심함\n- 식은땀, 창백함, 어지러움, 의식 저하\n- 구토, 혈변, 흑색변, 혈뇨\n- 호흡곤란, 가슴통증 동반\n- 고열(38도 이상) 또는 오한\n\n### 3. 현재 할 수 있는 조치\n- **금식**: 원인 모를 복통이 있을 때는 음식이나 물 섭취를 일시적으로 중단하는 것이 좋습니다.\n- **안정**: 무리한 움직임을 피하고, 편안한 자세로 휴식하세요.\n- **체온 측정**: 열이 있는지 확인하세요.\n- **증상 기록**: 통증의 위치, 양상, 지속 시간, 동반 증상(구토, 설사, 소변 이상 등)을 기록해두면 진료에 도움이 됩니다.\n\n### 4. 병원 방문이 필요한 경우\n- 통증이 1~2시간 이상 지속되거나 점점 심해질 때\n- 위의 위험 신호가 있을 때\n- 이전에 경험하지 못한 새로운 통증일 때\n- 만성질환(당뇨, 심장질환 등)이 있는 경우\n\n### 5. 진료 시 예상 검사\n- 혈액

In [14]:
prompt = f"""
                너는 사용자의 질문에 똑똑하고 정확하게 대답해주는 도우미야.
                포카라에 대해서 설명해줘
                """

In [15]:
response = invoke(prompt).choices[0].message.content

In [17]:
print(response)

네, 포카라(Pokhara)에 대해 설명해드릴게요!

**포카라**는 네팔 중서부에 위치한 도시로, 카트만두에 이어 네팔에서 두 번째로 큰 도시입니다. 해발 약 800m에 자리잡고 있으며, 히말라야 산맥의 아름다운 풍경과 자연 경관으로 매우 유명합니다.

### 주요 특징

1. **관광 명소**
   - **페와 호수(Phewa Lake)**: 포카라에서 가장 유명한 호수로, 배를 타고 호수 한가운데에 있는 바라히 사원(Barahi Temple)까지 갈 수 있습니다.
   - **사랑코트(Sarangkot)**: 일출과 히말라야 산맥(특히 마차푸차레와 안나푸르나 봉우리)을 감상할 수 있는 전망대로 유명합니다.
   - **데이비스 폭포(Davis Falls)**: 강물이 지하로 빨려 들어가는 독특한 폭포입니다.
   - **세계 평화 파고다(World Peace Pagoda)**: 멋진 전망과 평화로운 분위기를 즐길 수 있는 불교 사원입니다.

2. **등산과 트레킹의 관문**
   - 포카라는 안나푸르나(Annapurna) 트레킹의 출발점으로, 세계 각지의 트레커들이 모이는 곳입니다.
   - 패러글라이딩, 래프팅, 산악자전거 등 다양한 액티비티도 인기가 많아요.

3. **자연 경관**
   - 히말라야의 눈 덮인 봉우리와 푸른 호수, 울창한 숲이 어우러져 그림 같은 풍경을 자랑합니다.

4. **기후**
   - 포카라는 카트만두보다 온화하고, 연중 대부분 날씨가 좋습니다.

### 요약
포카라는 히말라야 산맥의 아름다운 풍경, 트레킹과 다양한 액티비티, 그리고 평화로운 분위기로 유명한 네팔의 대표적인 관광 도시입니다. 자연을 좋아하거나, 트레킹과 액티비티를 즐기고 싶은 분들에게 꼭 추천하는 여행지입니다!
